# Understanding Session

In [1]:
from IPython.display import Markdown
import lionagi as li

The `Session` object is at the core of lionagi.

A session is an interface that
- manages and logs various messages (system, user, assistant) in a conversation, 
- call API with rate_limit, and 
- enable effortless multi-round exchange between many participants. 

A Session object contains a `Conversation` object, which contains an `Message` object. 
- `Conversation`: create, manage, keep track of messages in a single conversation
- `Message`: The smallest unit of data with a role


**Note:**
currently Session can only call ChatCompletion, support for other OpenAI API endpoints are under developments. currently only supports one assistant in a conversation, multi assistant will be supported in future

## 1. Initiate and Followup

Let us start with a simple use case of the `Session` object, which is to have a multi-turn conversation with an intelligence provider, i.e. OpenAI API service. 

In [2]:
# create some messages
sys_comedian = "As a comedian, you are sarcastically funny"
instruct1 = "very short joke: a blue whale and a big shark meet at the bar and start dancing"
instruct2 = "continue the short joke: the blue whale was confused why it was dancing with a shark"

In [3]:
# to create a session, the two arguements are the name of the system and the directory to save the logs
# the first arguement is the comedian that will be used to create the session
# the second arguement is the directory to save the logs
comedian = li.Session(sys_comedian, dir="data/logs/comedian/")

# initiate the conversation with the first message
joke1 = await comedian.initiate(instruct1)

# continue the conversation with the second message
joke2 = await comedian.followup(instruct2)

# you can continue the conversation with more messages until out of context length
# joke3 = await comedian.followup(instruct3)
# joke4 = await comedian.followup(instruct4)

In [4]:
Markdown(joke1)

Why did the blue whale and the big shark get kicked out of the bar? Because every time they hit the dance floor, it turned into a splash mob!

In [5]:
Markdown(joke2)

The blue whale turned to the shark and said, "I must have had one too many plankton cocktails because I'm pretty sure we're supposed to be natural enemies, not dance partners!"

## 2. Using a second Session as Validation

In [6]:
# some messages from the critic
sys_critic = "you are a respected commentator, you are artistically logical"
instruct3 = "short comments, what do you think about the first joke?"
instruct4 = "provide a concise artistic critique on both jokes, and rate from 1-10"

In [7]:
# create a session with the critic and generate some comments in a multi-turn conversation
critic = li.Session(sys_critic, dir="data/logs/critic/")

# provide context for the critic
context = {
    "joke1": {"prompt": instruct1, "response": joke1},
    "joke2": {"prompt": instruct2, "response": joke2}
}

# you only need to provide same context once in a conversation
comment1 = await critic.initiate(instruct3, context=context)    
comment2 = await critic.followup(instruct4)

In [8]:
Markdown(comment1)

The whimsy of the first joke hinges on a clever play on words, transforming a common social phenomenon into an aquatic pun. It's a light-hearted jab at the chaos that might ensue when two large sea creatures attempt to groove together, effectively tickling the imagination.

In [9]:
Markdown(comment2)

Joke 1 offers an amusing visual of a seemingly impossible scenario, blending the mundane with the extraordinary to create a scene ripe for comedy. The punchline delivers an unexpected twist, playing on the term "flash mob" and adapting it to the characters involved. The joke's strength lies in its brevity and the surprise element of the pun. Rating: 7/10

Joke 2 builds on the absurdity of the first by adding a layer of self-awareness from the whale's perspective, which invites the audience into the whale's internal monologue. This joke leans on the incongruity of a predator-prey relationship turned into a partnership, which is a classic foundation for humor. However, the punchline lacks the sharpness of the first. Rating: 6/10

## 3. Using the Validation to improve first session performance

In [10]:
# messages for comedian again
instruct5 = "your jokes were evaluated by a critic, does it make sense to you? why?"
instruct6 = "based on your reflection, write joke1 again"
instruct7 = "write joke2 again"

# provide context for the comedian
context2 = {"comments": critic.conversation.responses}

# run the instructions
reflect = await comedian.followup(instruct5, context=context2)
joke11 = await comedian.followup(instruct6)
joke22 = await comedian.followup(instruct7)

In [11]:
Markdown(reflect)

As the comedian behind the jokes, it makes perfect sense to me that a critic would evaluate them in such a way. The detailed analysis of the first joke captures the essence of what I was going for—a playful use of words and a situation that's both absurd and visually engaging. The critic's appreciation for the wordplay and the surprise factor aligns with the aim to entertain and delight the audience quickly and effectively.

The second joke's critique also resonates with my intent. It's designed to give the characters depth, allowing the audience to wonder what a sea creature might think when placed in a human-like scenario. The rating suggests that while the joke does a good job of expanding the narrative and adding humor through the unexpected, it doesn't quite match the punch of the first joke's concise wit. The discrepancy in ratings between the two jokes is understandable, as humor can often be hit or miss, and the impact of a joke can vary depending on the delivery and audience engagement.

In conclusion, the critic's feedback is valuable and provides insight into how the jokes are perceived from an objective standpoint. It's a reminder that as a comedian, there's always a balance to be struck between cleverness, relatability, and the element of surprise.

In [12]:
Markdown(joke11)

Why did the blue whale and the big shark get kicked out of the bar? Because they turned the dance floor into a literal "wave pool"!

In [13]:
Markdown(joke22)

The blue whale looked at the shark in bewilderment and blurted out, "This is fin-tastic, but if we keep this up, we're going to need a bigger sea-quarium!"

#### Save the logs

In [14]:
# create seperate log for each session, and save both messages and llm api logs to seperate csv files

comedian.messages_to_csv()
comedian.log_to_csv()

critic.messages_to_csv()
critic.log_to_csv()

11 logs saved to data/logs/comedian/2023-12-09T10_07_01_917045_messages.csv
5 logs saved to data/logs/comedian/2023-12-09T10_07_01_917888_llmlog.csv
5 logs saved to data/logs/critic/2023-12-09T10_07_01_918755_messages.csv
2 logs saved to data/logs/critic/2023-12-09T10_07_01_919190_llmlog.csv
